# Анализ токсичности комментариев

**Заказчик.** Интернет-магазин «Викишоп»

**Цель Заказчика.** Получить инструмент, который будет искать токсичные комментарии пользователей и отправлять их на модерацию.

**Цель исследования.** Обучить модель классифицировать комментарии на позитивные и негативные со значением метрики качества F1 не меньше 0.75.

**Задачи:**

- Загрузить и подготовить данные.
- Обучить разные модели.
- Сделать выводы.

**Входные данные от Заказчика.** Файл в формате .csv с текстами комментариев и классификацией их токсичности

**Ожидаемый результат.** Построена модель для классификации комментариев на позитивные и негативные со значением метрики качества F1 более 0.75


Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

**Заказчик.** Интернет-магазин «Викишоп»

**Цель Заказчика.** Получить инструмент, который будет искать токсичные комментарии пользователей и отправлять их на модерацию.

**Цель исследования.** Обучить модель классифицировать комментарии на позитивные и негативные со значением метрики качества F1 не меньше 0.75.

**Задачи:**

- Загрузить и подготовить данные.
- Обучить разные модели.
- Сделать выводы.

**Входные данные от Заказчика.** Файл в формате .csv с текстами комментариев и классификацией их токсичности

**Ожидаемый результат.** Построена модель для классификации комментариев на позитивные и негативные со значением метрики качества F1 более 0.75


## Подготовка

**Признаки**

- `text` — коментарии пользователей

**Цеоевой признак**

- `toxic` — классификация комментариев

In [36]:
# Импотрт библиотек
import pandas as pd
import numpy as np
from numpy.random import RandomState

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import cross_val_score # кросс-вылидация
from sklearn.model_selection import GridSearchCV # кросс-вылидация
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.dummy import DummyRegressor
from sklearn.metrics import f1_score

from sklearn import pipeline
from sklearn.feature_extraction.text import TfidfTransformer

from time import time
from pymystem3 import Mystem

In [37]:
# Введем константы
state = RandomState(12345)

In [38]:
try:
    df = pd.read_csv('C:\\Users\\Aser\\Первый ноутбук\\Projects\\data\\toxic_comments.csv')

except FileNotFoundError:
    df = pd.read_csv('/datasets/toxic_comments.csv')

In [39]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [40]:
df.shape

(159571, 2)

In [41]:
# Изучим баланс классов
df['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

In [42]:
corpus = df['text']

In [43]:
len(corpus)

159571

In [44]:
# Функция для лемматизации текста
def lemmatize(text):
    l = WordNetLemmatizer()
    word_list = nltk.word_tokenize(text)
    lemm_text = ' '.join([l.lemmatize(w) for w in word_list])
    return lemm_text

In [45]:
corpus = pd.Series(corpus)

Проверим скорость лемитизации на 100 твитах 

In [46]:
corpus_t = corpus[:100]

In [47]:
%%time
corpus_lem_func = corpus_t.apply(lemmatize)
print(corpus_lem_func.shape)

(100,)
CPU times: user 85.8 ms, sys: 19 µs, total: 85.8 ms
Wall time: 88.5 ms


Проведем лемматизацию после очистки текста

In [48]:
# Функция для очистки текста
def clear_text(text):
    t_str = re.sub(r'[^a-zA-Z \']', " ", text)
    t_str = " ".join(t_str.split())
    return t_str

In [49]:
corpus = corpus.apply(clear_text)

In [50]:
len(corpus)

159571

In [51]:
%%time
corpus_lem = corpus.apply(lemmatize)
print(corpus_lem.shape)

(159571,)
CPU times: user 1min 18s, sys: 199 ms, total: 1min 18s
Wall time: 1min 18s


## Обучение

Очистка от стоп слов и построение матрицы

In [52]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Разделим данные на обучающую, валидационную и тестовыую выборки в соотношении `60:20:20` 

In [53]:
# Целевые прзнаки
tox = df['toxic']

In [54]:
corp_train, corp_1, tox_train, tox_1 = train_test_split(corpus_lem, tox, 
                                             test_size=.4,
                                             random_state=state)

corp_valid, corp_test, tox_valid, tox_test = train_test_split(corp_1, tox_1, 
                                                   test_size=.5,
                                                   random_state=state)

y_train = np.array(tox_train)
y_valid = np.array(tox_valid)
y_test = np.array(tox_test)

print('Число ответов обучающей выборки', y_train.shape)
print('Число ответов валидационной выборки', y_valid.shape)
print('Число ответов тестовой выборки', y_test.shape)

Число ответов обучающей выборки (95742,)
Число ответов валидационной выборки (31914,)
Число ответов тестовой выборки (31915,)


In [55]:
# Обучим векторизатор
count_tfidf_lem = TfidfVectorizer(stop_words=stop_words)
X_train = count_tfidf_lem.fit_transform(corp_train)
print('Размер обучающей выборки', X_train.shape, 'число ответов', y_train.shape)

Размер обучающей выборки (95742, 121746) число ответов (95742,)


Обучение методом `Логистической регрессии` 

In [56]:
%%time
model_reg = LogisticRegression()
model_reg.fit(X_train, y_train)

X_valid = count_tfidf_lem.transform(corp_valid)
pred_reg = model_reg.predict(X_valid)

answer_reg = f1_score(y_valid, pred_reg)
print(f'Логистическая регрессия достигает значение метрики F1 {answer_reg:.4f}')

/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Логистическая регрессия достигает значение метрики F1 0.7224
CPU times: user 25.4 s, sys: 21.9 s, total: 47.3 s
Wall time: 47.5 s


**Уменьшение выборки**

Учитывая текущее соотношение целевого признака `zero-0.9 : one-0.1`, **уменьшим количество позитивных комментариев в девять раз**

In [57]:
# Разделим выборку

features_one = corp_train[tox_train==1]
features_zero = corp_train[tox_train==0]
target_one = tox_train[tox_train==1]
target_zero = tox_train[tox_train==0]

In [58]:
# Уменьшим выборку используя уже разделенные таблицы
zero_down = 0.112
features_downsampled = pd.concat([features_zero.sample(frac=zero_down, random_state=state)] + [features_one])
target_downsampled = pd.concat([target_zero.sample(frac=zero_down, random_state=state)] + [target_one])

# Перемешаем данные
features_downsampled, target_downsampled = shuffle(features_downsampled, 
                                                   target_downsampled,
                                                   random_state=state)

print('features_downsampled', features_downsampled.shape)
print('target_downsampled', target_downsampled.shape)


features_downsampled (19384,)
target_downsampled (19384,)


In [59]:
# Преобразуем данные

X_train_down = count_tfidf_lem.transform(features_downsampled)
y_train_down = np.array(target_downsampled)

print('Размер обучающей выборки', X_train_down.shape, 'число ответов', y_train_down.shape)

Размер обучающей выборки (19384, 121746) число ответов (19384,)


Обучение методом `Логистической регрессии` на сбалансированной (уменьшенной) выборке

In [60]:
%%time

model_reg_down = LogisticRegression()
model_reg_down.fit(X_train_down, y_train_down)
pred_reg_down = model_reg_down.predict(X_valid)
answer_reg_down = f1_score(y_valid, pred_reg_down)

print(f'Логистическая регрессия на сбалансированной вниз выборке достигает значение метрики F1 {answer_reg_down:.4f}')

Логистическая регрессия на сбалансированной вниз выборке достигает значение метрики F1 0.6872
CPU times: user 11.8 s, sys: 12.9 s, total: 24.7 s
Wall time: 24.7 s


**Увеличение выборки**

Учитывая текущее соотношение целевого признака `zero-0.9 : one-0.1`, увеличим количество положительных признаков в 9 раз

In [61]:
# Увеличим положительный класс и объединим таблицы
one_height = 9
features_upsampled = pd.concat([features_zero] + [features_one] * one_height)
target_upsampled = pd.concat([target_zero] + [target_one] * one_height)

# Перемешаем данные
features_upsampled = shuffle(features_upsampled, random_state=state)
target_upsampled = shuffle(target_upsampled, random_state=state)

print('features_upsampled', features_upsampled.shape)
print('target_upsampled', target_upsampled.shape)

features_upsampled (173766,)
target_upsampled (173766,)


In [62]:
# Преобразуем данные

X_train_up = count_tfidf_lem.transform(features_upsampled)
y_train_up = np.array(target_upsampled)

print('Размер обучающей выборки', X_train_up.shape, 'число ответов', y_train_up.shape)

Размер обучающей выборки (173766, 121746) число ответов (173766,)


Обучение методом `Логистической регрессии` на сбалансированной (увеличенной) выборке

In [63]:
%%time

model_reg_up = LogisticRegression()
model_reg_up.fit(X_train_up, y_train_up)
pred_reg_up = model_reg_up.predict(X_valid)
answer_reg_up = f1_score(y_valid, pred_reg_up)

print(f'Логистическая регрессия на сбалансированной вверх выборке достигает значение метрики F1 {answer_reg_up:.4f}')

Логистическая регрессия на сбалансированной вверх выборке достигает значение метрики F1 0.1775
CPU times: user 25.1 s, sys: 24.5 s, total: 49.6 s
Wall time: 49.7 s


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


**Балансирование выборки**

Учитывая 
- текущее соотношение целевого признака `zero-0.9 : one-0.1`, 
- низкое значеличим метрикипри уменьшении выборки
- невозможность работы с увеличенной выборкрй

1. Увеличим количество положительных признаков в 2 раза
2. Уменьшим количество отрицательных признаков в 4 раза

In [64]:
# Увеличим положительный класс, уменьшим отрицательный и объединим таблицы
one_height = 2
zero_down = 0.25

features_ok = pd.concat([features_zero.sample(frac=zero_down, random_state=state)] + [features_one] * one_height)
target_ok = pd.concat([target_zero.sample(frac=zero_down, random_state=state)] + [target_one] * one_height)

# Перемешаем данные
features_ok = shuffle(features_ok, random_state=state)
target_ok = shuffle(target_ok, random_state=state)

print('features_ok', features_ok.shape)
print('target_ok', target_ok.shape)

features_ok (41003,)
target_ok (41003,)


In [65]:
# Преобразуем данные

X_train_ok = count_tfidf_lem.transform(features_ok)
y_train_ok = np.array(target_ok)

print('Размер обучающей выборки', X_train_ok.shape, 'число ответов', y_train_ok.shape)

Размер обучающей выборки (41003, 121746) число ответов (41003,)


Обучение методом `Логистической регрессии` на сбалансированной с двух сторон выборке

In [66]:
%%time

model_reg_ok = LogisticRegression()
model_reg_ok.fit(X_train_ok, y_train_ok)
pred_reg_ok = model_reg_ok.predict(X_valid)
answer_reg_ok = f1_score(y_valid, pred_reg_ok)

print(f'Логистическая регрессия на двусторонне сбалансированной выборке достигает значение метрики F1 {answer_reg_ok:.4f}')

Логистическая регрессия на двусторонне сбалансированной выборке достигает значение метрики F1 0.1629
CPU times: user 19.6 s, sys: 22.6 s, total: 42.2 s
Wall time: 42.4 s


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Подберем параметр `С` для паервоначальной модели `Логистической регресии`

In [67]:
%%time

list_c = [0.01, 0.1, 5]
best_model_reg_c = None
best_answer_reg_c = 0

for i in list_c:
    model_reg_c = LogisticRegression(C=i)
    model_reg_c.fit(X_train, y_train)

    pred_reg_c = model_reg_c.predict(X_valid)
    answer_reg_c = f1_score(y_valid, pred_reg_c)
    print(f'Логистическая регрессия с параметром С {i} достигает значение метрики F1 {answer_reg_c:.4f}')
    
    if best_answer_reg_c < answer_reg_c:
        best_answer_reg_c = answer_reg_c
        best_model_reg_c = model_reg_c

Логистическая регрессия с параметром С 0.01 достигает значение метрики F1 0.0374
Логистическая регрессия с параметром С 0.1 достигает значение метрики F1 0.4780
Логистическая регрессия с параметром С 5 достигает значение метрики F1 0.7674
CPU times: user 36 s, sys: 36.7 s, total: 1min 12s
Wall time: 1min 12s


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Лучший результат показала первоначальная модель логистической регресии с параметтром `С=5`.

Обучение методом `Решающего дерева`

In [68]:
# Подберем лучшие параметры Решающего дерева методом GridSearchCV
clf = DecisionTreeClassifier(random_state=state)
parametrs = {'max_depth': range(7, 11)}

model_tree_cv = GridSearchCV(clf, parametrs, cv=5)
model_tree_cv.fit(X_train, y_train)
best_depth_tree = model_tree_cv.best_params_.get('max_depth')

print(f'Лучший параметр глубины - {best_depth_tree}')

# Обучим и проверим модель с подобранными параметрами
model_tree = DecisionTreeClassifier(max_depth=best_depth_tree, random_state=state)
model_tree.fit(X_train, y_train)
pred_tree = model_tree.predict(X_valid)
answer_tree = f1_score(y_valid, pred_tree)
print(f'Решающее дерево с глубиной {best_depth_tree} достигает значение метрики F1 {answer_tree:.4f}')

Лучший параметр глубины - 10
Решающее дерево с глубиной 10 достигает значение метрики F1 0.5906


Обучение методом `Решающего дерева` применив `pipeline` при `кросс-валидации`

In [69]:
%%time

# Создададим pipeline

pipe_tree = pipeline.Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', DecisionTreeClassifier(random_state=state))])

parametrs = {'clf__max_depth': range(5, 11)}
model_tree_cv_pl = GridSearchCV(pipe_tree, parametrs, cv=5)
model_tree_cv_pl.fit(corp_train, tox_train)
best_depth_tree_pl = model_tree_cv_pl.best_params_.get('clf__max_depth')

print(f'Лучший параметр глубины - {best_depth_tree_pl}')

# Обучим и проверим модель с подобранными параметрами
model_tree_pl = DecisionTreeClassifier(max_depth=best_depth_tree_pl, random_state=state)
model_tree_pl.fit(X_train, y_train)
pred_tree_pl = model_tree_pl.predict(X_valid)
answer_tree_pl = f1_score(y_valid, pred_tree_pl)
print(f'Решающее дерево с глубиной {best_depth_tree_pl} достигает значение метрики F1 {answer_tree_pl:.4f}')

Лучший параметр глубины - 10
Решающее дерево с глубиной 10 достигает значение метрики F1 0.5901
CPU times: user 4min 18s, sys: 0 ns, total: 4min 18s
Wall time: 4min 18s


Обучение методом `Случайного леса`

In [70]:
# Подберем лучшие параметры Случайного леса методом GridSearchCV
clf = RandomForestClassifier(max_depth=best_depth_tree, random_state=state)
parametrs = {'n_estimators': range(20, 51, 10)}

model_forest_cv = GridSearchCV(clf, parametrs, cv=5)
model_forest_cv.fit(X_train, y_train)
best_est_forest = model_forest_cv.best_params_.get('n_estimators')

print(f'Лучший параметр числа деревьев - {best_est_forest}')

# Обучим и проверим модель с подобранными параметрами
model_forest = RandomForestClassifier(max_depth=best_depth_tree, n_estimators=best_est_forest, random_state=state)
model_forest.fit(X_train, y_train)
pred_forest = model_forest.predict(X_valid)
answer_forest = f1_score(y_valid, pred_tree)
print(f'случайный лес с числом деревьев {best_est_forest} достигает значение метрики F1 {answer_forest:.4f}')

Лучший параметр числа деревьев - 20
случайный лес с числом деревьев 20 достигает значение метрики F1 0.5906


Обучение методом `Случайного леса` применив `pipeline` при `кросс-валидации`

In [71]:
%%time

# Создададим pipeline

pipe_forest = pipeline.Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', RandomForestClassifier(max_depth=best_depth_tree_pl, random_state=state))])

parametrs = {'clf__n_estimators': range(20, 51, 10)}
model_forest_cv_pl = GridSearchCV(pipe_forest, parametrs, cv=5)
model_forest_cv_pl.fit(corp_train, tox_train)
best_est_forest_pl = model_forest_cv_pl.best_params_.get('clf__n_estimators')

print(f'Лучший параметр числа деревьев - {best_est_forest_pl}')

# Обучим и проверим модель с подобранными параметрами
model_forest_pl = RandomForestClassifier(max_depth=best_depth_tree_pl, 
                                         n_estimators=best_est_forest_pl, 
                                         random_state=state)
model_forest_pl.fit(X_train, y_train)
pred_forest_pl = model_forest_pl.predict(X_valid)
answer_forest_pl = f1_score(y_valid, pred_tree_pl)
print(f'случайный лес с числом деревьев {best_est_forest_pl} достигает значение метрики F1 {answer_forest_pl:.4f}')


Лучший параметр числа деревьев - 20
случайный лес с числом деревьев 20 достигает значение метрики F1 0.5901
CPU times: user 2min 3s, sys: 0 ns, total: 2min 3s
Wall time: 2min 3s


## Выводы

Проверим лучшую модкль `Логистической регрессии` на тестовой выборке

In [72]:
X_test = count_tfidf_lem.transform(corp_test)
pred_reg_test = best_model_reg_c.predict(X_test)
answer_reg_test = f1_score(y_test, pred_reg_test)

print(f'Логистическая регрессия на тестовой выборке достигает значение метрики F1 {answer_reg_test:.4f}')

Логистическая регрессия на тестовой выборке достигает значение метрики F1 0.7634


Параметры для методов Решающего дерева и Случайного леса было подобраны с помощью крос-валдации. Показателя метрики F1 у этих моделей крайне низкие (0.5823 и 0.5823 соответственно), что не позволяет применять эти модели для работы.

Лучший показатель метрики F1 на валидационной выборке у модели логистической регрессии составляет 0.7113

Уменьшение выборки (повышение баланма) не дало результата. Показатель метрики F1 достиг резултата 0,6903

При подборе параметра С (при С=5) на перовночальной моделе удалось достигнуть значения показателя метрики F1 0.7681

Для тестовой выборки этой модели показатель метрики F1 достигает результата 0.7675

Предложенная модель логистической регрессии позволила преодолеть целевое значение показателя метрики F1 0,75. Максимально достигнутый результат на тестовой выборке составляет **0.7671**
